In [ ]:
import torch.optim as optim

# Initialize models, optimizers, and criterion
generator = Generator().to(device)
discriminator = Discriminator().to(device)
criterion = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

num_epochs = 100

for epoch in range(num_epochs):
    for i, data in enumerate(dataloader):
        real_voxels = data['voxels'].to(device)  # Real 3D voxel grids
        real_labels = torch.ones(real_voxels.size(0), 1).to(device)
        fake_labels = torch.zeros(real_voxels.size(0), 1).to(device)
        images = data['images'].to(device)  # 2D images

        # Train Discriminator
        optimizer_D.zero_grad()
        outputs = discriminator(real_voxels, images)
        d_loss_real = criterion(outputs, real_labels)
        d_loss_real.backward()

        z = torch.randn(images.size(0), latent_dim).to(device)  # Random noise
        fake_voxels = generator(images, z)
        outputs = discriminator(fake_voxels.detach(), images)
        d_loss_fake = criterion(outputs, fake_labels)
        d_loss_fake.backward()
        optimizer_D.step()

        # Train Generator
        optimizer_G.zero_grad()
        outputs = discriminator(fake_voxels, images)
        g_loss = criterion(outputs, real_labels)  # Trick D into thinking generated is real
        g_loss.backward()
        optimizer_G.step()

        if i % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(dataloader)}], D Loss: {d_loss_real.item() + d_loss_fake.item():.4f}, G Loss: {g_loss.item():.4f}")